<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2E_Bloque_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones')
from analisis_fase_2e_config import inicializar_fase2e
estado = inicializar_fase2e()

[00:06:36] INFO - ============================================================


INFO:Fase2E_Init:============================================================


[00:06:36] INFO - INICIALIZANDO FASE 2E - VISUALIZACIONES


INFO:Fase2E_Init:INICIALIZANDO FASE 2E - VISUALIZACIONES


[00:06:36] INFO - ============================================================


...

INFO:Fase2E_Init:  anova_por_modelo: OK


[00:06:36] INFO -   indice_maestro: OK


INFO:Fase2E_Init:  indice_maestro: OK


[00:06:36] INFO - Archivos disponibles: 10/10


INFO:Fase2E_Init:Archivos disponibles: 10/10


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2E - BLOQUE 2: DASHBOARDS INDIVIDUALES POR FOTOGRAFIA
================================================================================

Trabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion
                        en Fotografia de Retrato

Autor: Jesus L.
Universidad: Universitat Oberta de Catalunya (UOC)
Master: Data Science
Fecha: Diciembre 2025

Descripcion:
    Generacion de 4 dashboards individuales con analisis detallado por fotografia.

Uso:
    from analisis_fase_2e_bloque2 import ejecutar_bloque2
    resultados = ejecutar_bloque2()
================================================================================
"""

'\n================================================================================\nFASE 2E - BLOQUE 2: DASHBOARDS INDIVIDUALES POR FOTOGRAFIA\n================================================================================\n\nTrabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion\n                        en Fotografia de Retrato\n\nAutor: Jesus L. (Iesus)\nUniversidad: Universitat Oberta de Catalunya (UOC)\nMaster: Data Science\nFecha: Diciembre 2025\n\nDescripcion:\n    Generacion de 4 dashboards individuales con analisis detallado por fotografia.\n\nUso:\n    from analisis_fase_2e_bloque2 import ejecutar_bloque2\n    resultados = ejecutar_bloque2()\n================================================================================\n'

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
from pathlib import Path
from typing import Dict, List, Optional

try:
    from analisis_fase_2e_config import (
        RUTAS, ARCHIVOS, FIGSIZE,
        PALETTE_MODELS, ORDEN_MODELOS, NOMBRES_MODELOS,
        FOTOS_SELECCIONADAS,
        configurar_logging, aplicar_estilo, guardar_figura,
        agregar_columna_modelo
    )
except ImportError:
    exec(open('03_analisis_fase_2e_config.py').read())

In [ ]:
def identificar_fotos_extremas(df: pd.DataFrame) -> Dict[str, str]:
    """Identifica fotografias con mayor y menor IoU promedio."""
    # Detectar nombre de columna de foto
    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    df_agg = df.groupby([col_foto, 'modelo_norm'])['iou'].max().reset_index()
    df_foto = df_agg.groupby(col_foto)['iou'].mean().reset_index()
    df_foto = df_foto.sort_values('iou', ascending=False)

    return {
        'facil': df_foto.iloc[0][col_foto],
        'dificil': df_foto.iloc[-1][col_foto],
        'iou_facil': df_foto.iloc[0]['iou'],
        'iou_dificil': df_foto.iloc[-1]['iou']
    }


def obtener_metricas_foto(df: pd.DataFrame, foto_id: str) -> pd.DataFrame:
    """Obtiene metricas de una fotografia (mejor config por modelo)."""
    # Detectar nombre de columna de foto
    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    df_foto = df[df[col_foto] == foto_id].copy()
    if len(df_foto) == 0:
        return pd.DataFrame()
    idx = df_foto.groupby('modelo_norm')['iou'].idxmax()
    return df_foto.loc[idx]


def obtener_info_cluster(df_clusters: pd.DataFrame, foto_id: str) -> Dict:
    """Obtiene informacion de cluster para una fotografia."""
    if df_clusters is None:
        return {'cluster': 'N/A', 'nombre': 'No disponible'}

    # Detectar nombre de columna de foto
    col_foto = 'codigo_foto' if 'codigo_foto' in df_clusters.columns else 'foto_id'

    row = df_clusters[df_clusters[col_foto] == foto_id]
    if len(row) == 0:
        return {'cluster': 'N/A', 'nombre': 'No disponible'}
    cluster = row.iloc[0].get('cluster', 'N/A')
    nombres = {0: 'Facil', 1: 'Medio', 2: 'Dificil'}
    return {'cluster': cluster, 'nombre': nombres.get(cluster, str(cluster))}

In [ ]:
def crear_dashboard(
    df: pd.DataFrame,
    foto_id: str,
    titulo: str,
    df_clusters: pd.DataFrame = None,
    output_dir: Path = None,
    logger=None
) -> plt.Figure:
    """Crea dashboard completo para una fotografia."""
    if logger:
        logger.info(f"Generando dashboard: {foto_id}")

    df_foto = obtener_metricas_foto(df, foto_id)

    if len(df_foto) == 0:
        fig, ax = plt.subplots(figsize=FIGSIZE['dashboard'])
        ax.text(0.5, 0.5, f'Sin datos para {foto_id}', ha='center', va='center')
        return fig

    fig = plt.figure(figsize=FIGSIZE['dashboard'])
    gs = gridspec.GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.25)

    # Panel A: Bar chart IoU
    ax_bar = fig.add_subplot(gs[0, 0])
    modelos_disp = [m for m in ORDEN_MODELOS if m in df_foto['modelo_norm'].values]
    valores_iou = [df_foto[df_foto['modelo_norm'] == m]['iou'].values[0] for m in modelos_disp]
    colores = [PALETTE_MODELS.get(m, '#7F8C8D') for m in modelos_disp]
    labels = [NOMBRES_MODELOS.get(m, m) for m in modelos_disp]

    y_pos = np.arange(len(modelos_disp))
    bars = ax_bar.barh(y_pos, valores_iou, color=colores, edgecolor='white', height=0.6)
    ax_bar.set_yticks(y_pos)
    ax_bar.set_yticklabels(labels, fontsize=10)
    ax_bar.set_xlabel('IoU', fontsize=11)
    ax_bar.set_title('Rendimiento por Modelo', fontsize=12, fontweight='bold')
    ax_bar.set_xlim(0, 1.05)
    ax_bar.invert_yaxis()
    for bar, val in zip(bars, valores_iou):
        ax_bar.text(val + 0.02, bar.get_y() + bar.get_height()/2, f'{val:.4f}', va='center', fontsize=9)
    ax_bar.axvline(x=0.5, color='gray', linestyle=':', alpha=0.5)
    ax_bar.xaxis.grid(True, linestyle='--', alpha=0.5)

    # Panel B: Tabla de metricas
    ax_tabla = fig.add_subplot(gs[0, 1])
    ax_tabla.axis('off')
    metricas = ['iou', 'dice', 'precision', 'recall']
    metricas_disp = [m for m in metricas if m in df_foto.columns]

    tabla_data = [['Modelo'] + [m.upper() for m in metricas_disp]]
    for modelo in modelos_disp:
        row = df_foto[df_foto['modelo_norm'] == modelo].iloc[0]
        fila = [NOMBRES_MODELOS.get(modelo, modelo)]
        for met in metricas_disp:
            val = row.get(met, np.nan)
            fila.append(f'{val:.4f}' if not pd.isna(val) else 'N/A')
        tabla_data.append(fila)

    tabla = ax_tabla.table(cellText=tabla_data[1:], colLabels=tabla_data[0],
                           loc='center', cellLoc='center', colColours=['#E8E8E8'] * len(tabla_data[0]))
    tabla.auto_set_font_size(False)
    tabla.set_fontsize(9)
    tabla.scale(1.2, 1.5)
    ax_tabla.set_title('Metricas de Segmentacion', fontsize=12, fontweight='bold', y=0.95)

    # Panel C: Radar chart
    ax_radar = fig.add_subplot(gs[1, 0], polar=True)
    if len(metricas_disp) >= 3:
        num_vars = len(metricas_disp)
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        angles += angles[:1]
        for modelo in modelos_disp:
            row = df_foto[df_foto['modelo_norm'] == modelo].iloc[0]
            valores = [row.get(m, 0) for m in metricas_disp]
            valores += valores[:1]
            color = PALETTE_MODELS.get(modelo, '#7F8C8D')
            ax_radar.plot(angles, valores, 'o-', linewidth=1.5, color=color,
                         label=NOMBRES_MODELOS.get(modelo, modelo), markersize=4)
            ax_radar.fill(angles, valores, alpha=0.1, color=color)
        ax_radar.set_xticks(angles[:-1])
        ax_radar.set_xticklabels([m.upper() for m in metricas_disp], fontsize=9)
        ax_radar.set_ylim(0, 1)
        ax_radar.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize=8)
    ax_radar.set_title('Perfil de Metricas', fontsize=12, fontweight='bold', y=1.1)

    # Panel D: Informacion
    ax_info = fig.add_subplot(gs[1, 1])
    ax_info.axis('off')
    iou_prom = df_foto['iou'].mean()
    iou_max = df_foto['iou'].max()
    iou_min = df_foto['iou'].min()
    iou_std = df_foto['iou'].std()
    mejor = df_foto.loc[df_foto['iou'].idxmax(), 'modelo_norm']
    peor = df_foto.loc[df_foto['iou'].idxmin(), 'modelo_norm']
    info_cluster = obtener_info_cluster(df_clusters, foto_id)

    info_text = f"""
    FOTOGRAFIA: {foto_id}

    ESTADISTICAS IoU:
    Promedio:     {iou_prom:.4f}
    Maximo:       {iou_max:.4f}
    Minimo:       {iou_min:.4f}
    Desv. Std:    {iou_std:.4f}

    MODELOS:
    Mejor:  {NOMBRES_MODELOS.get(mejor, mejor)}
    Peor:   {NOMBRES_MODELOS.get(peor, peor)}

    CLUSTER: {info_cluster['nombre']}
    """
    ax_info.text(0.1, 0.9, info_text, transform=ax_info.transAxes, fontsize=10,
                verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='#F5F5F5', alpha=0.8))
    ax_info.set_title('Resumen Estadistico', fontsize=12, fontweight='bold')

    fig.suptitle(titulo, fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    return fig

In [ ]:
def viz_14_dashboard_foto_facil(df, df_clusters, output_dir, logger=None):
    """Dashboard de la fotografia con mayor IoU promedio."""
    fotos = identificar_fotos_extremas(df)
    titulo = f"Dashboard Fotografia Facil: {fotos['facil']}\n(IoU Promedio: {fotos['iou_facil']:.4f})"
    fig = crear_dashboard(df, fotos['facil'], titulo, df_clusters, output_dir, logger)
    guardar_figura(fig, 'viz_14_dashboard_foto_facil', output_dir, logger=logger)
    return fig


def viz_15_dashboard_foto_dificil(df, df_clusters, output_dir, logger=None):
    """Dashboard de la fotografia con menor IoU promedio."""
    fotos = identificar_fotos_extremas(df)
    titulo = f"Dashboard Fotografia Dificil: {fotos['dificil']}\n(IoU Promedio: {fotos['iou_dificil']:.4f})"
    fig = crear_dashboard(df, fotos['dificil'], titulo, df_clusters, output_dir, logger)
    guardar_figura(fig, 'viz_15_dashboard_foto_dificil', output_dir, logger=logger)
    return fig


def viz_16_dashboard_foto_rep1(df, df_clusters, output_dir, foto_id=None, logger=None):
    """Dashboard de fotografia representativa 1."""
    # Detectar nombre de columna de foto
    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    if foto_id is None:
        foto_id = FOTOS_SELECCIONADAS[0] if FOTOS_SELECCIONADAS else None
    if foto_id is None:
        return None
    df_foto = df[df[col_foto] == foto_id]
    iou_prom = df_foto.groupby('modelo_norm')['iou'].max().mean() if len(df_foto) > 0 else 0
    titulo = f"Dashboard Fotografia Representativa: {foto_id}\n(IoU Promedio: {iou_prom:.4f})"
    fig = crear_dashboard(df, foto_id, titulo, df_clusters, output_dir, logger)
    guardar_figura(fig, 'viz_16_dashboard_foto_rep1', output_dir, logger=logger)
    return fig


def viz_17_dashboard_foto_rep2(df, df_clusters, output_dir, foto_id=None, logger=None):
    """Dashboard de fotografia representativa 2."""
    # Detectar nombre de columna de foto
    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    if foto_id is None:
        foto_id = FOTOS_SELECCIONADAS[1] if len(FOTOS_SELECCIONADAS) > 1 else None
    if foto_id is None:
        return None
    df_foto = df[df[col_foto] == foto_id]
    iou_prom = df_foto.groupby('modelo_norm')['iou'].max().mean() if len(df_foto) > 0 else 0
    titulo = f"Dashboard Fotografia Representativa: {foto_id}\n(IoU Promedio: {iou_prom:.4f})"
    fig = crear_dashboard(df, foto_id, titulo, df_clusters, output_dir, logger)
    guardar_figura(fig, 'viz_17_dashboard_foto_rep2', output_dir, logger=logger)
    return fig

In [ ]:
def ejecutar_bloque2(
    ruta_metricas: Path = None,
    ruta_clusters: Path = None,
    output_dir: Path = None,
    fotos_representativas: List[str] = None
) -> Dict[str, any]:
    """Ejecuta todas las visualizaciones del Bloque 2."""
    logger = configurar_logging('Fase2E_Bloque2')

    logger.info("=" * 60)
    logger.info("FASE 2E - BLOQUE 2: DASHBOARDS INDIVIDUALES")
    logger.info("=" * 60)

    aplicar_estilo()

    if ruta_metricas is None:
        ruta_metricas = ARCHIVOS['metricas_fusionadas']
    if ruta_clusters is None:
        ruta_clusters = ARCHIVOS['clusters_fotografias']
    if output_dir is None:
        output_dir = RUTAS['output_bloque2']

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    logger.info("\nCargando datos...")
    try:
        df = pd.read_csv(ruta_metricas)
        df = agregar_columna_modelo(df)
        logger.info(f"Metricas cargadas: {len(df)} registros")
    except Exception as e:
        logger.error(f"Error cargando metricas: {e}")
        return {'error': str(e)}

    df_clusters = None
    try:
        if Path(ruta_clusters).exists():
            df_clusters = pd.read_csv(ruta_clusters)
            logger.info(f"Clusters cargados: {len(df_clusters)} fotografias")
    except Exception as e:
        logger.warning(f"Clusters no disponibles: {e}")

    if fotos_representativas is None:
        fotos_representativas = FOTOS_SELECCIONADAS[:2]

    fotos_extremas = identificar_fotos_extremas(df)
    logger.info(f"Foto facil: {fotos_extremas['facil']} (IoU: {fotos_extremas['iou_facil']:.4f})")
    logger.info(f"Foto dificil: {fotos_extremas['dificil']} (IoU: {fotos_extremas['iou_dificil']:.4f})")

    logger.info("\nGenerando dashboards...")
    resultados = {'figuras': [], 'errores': [], 'fotos_extremas': fotos_extremas}

    dashboards = [
        ('viz_14', lambda: viz_14_dashboard_foto_facil(df, df_clusters, output_dir, logger)),
        ('viz_15', lambda: viz_15_dashboard_foto_dificil(df, df_clusters, output_dir, logger)),
        ('viz_16', lambda: viz_16_dashboard_foto_rep1(df, df_clusters, output_dir,
                                                       fotos_representativas[0] if fotos_representativas else None, logger)),
        ('viz_17', lambda: viz_17_dashboard_foto_rep2(df, df_clusters, output_dir,
                                                       fotos_representativas[1] if len(fotos_representativas) > 1 else None, logger)),
    ]

    for nombre, func in dashboards:
        try:
            fig = func()
            if fig:
                resultados['figuras'].append(nombre)
                plt.close(fig)
        except Exception as e:
            logger.error(f"Error en {nombre}: {e}")
            resultados['errores'].append((nombre, str(e)))

    logger.info("\n" + "=" * 60)
    logger.info("RESUMEN BLOQUE 2")
    logger.info("=" * 60)
    logger.info(f"Dashboards generados: {len(resultados['figuras'])}/4")
    logger.info(f"Errores: {len(resultados['errores'])}")

    return resultados

In [ ]:
if __name__ == "__main__":
    resultados = ejecutar_bloque2()
    print(f"\nDashboards generados: {len(resultados.get('figuras', []))}")

[00:06:36] INFO - ============================================================


INFO:Fase2E_Bloque2:============================================================


[00:06:36] INFO - FASE 2E - BLOQUE 2: DASHBOARDS INDIVIDUALES


INFO:Fase2E_Bloque2:FASE 2E - BLOQUE 2: DASHBOARDS INDIVIDUALES


[00:06:36] INFO - ============================================================


...

[00:06:44] INFO - Dashboards generados: 4/4


INFO:Fase2E_Bloque2:Dashboards generados: 4/4


[00:06:44] INFO - Errores: 0


INFO:Fase2E_Bloque2:Errores: 0



Dashboards generados: 4
